In [156]:
from backend.db.products import product_db


await product_db.check_exist_and_initialize()


True

In [157]:
DIALOG_FORMAT = """
Dealer: "{dealer}"
Customer: "{customer}"
""".strip()


In [158]:
import itertools
res = product_db.iterate()

my_list = []
i = 0
async for product in res:
    i += 1
    if i > 50:
        break
    # print(product)
    my_list.append(product)

print(my_list)

[Product(name='Certified Pre-Owned 2016 Mercedes-Benz GLE 450 AMG   With Navigation', vehicle_type=<VehicleType.SUV: 'SUV'>, release_year=2016, price_in_usd=39998.0, is_used=True, powertrain_type=<Powertrain.ICE: 'ICE'>, num_seats=5, color='White', specs={'year': 2016, 'make': 'Mercedes-Benz', 'model': 'GLE-Class Coupe', 'trim': 'GLE450 AMG', 'body_type': 'SUV', 'vehicle_type': 'Truck', 'transmission': 'Automatic', 'drivetrain': '4WD', 'fuel_type': 'Premium Unleaded', 'engine': '3.0L V6', 'engine_size': 3, 'engine_block': 'V', 'doors': 5, 'cylinders': 6, 'made_in': 'USA', 'overall_height': '67.7', 'overall_length': '192.6', 'std_seating': '5', 'highway_mpg': 23, 'city_mpg': 17, 'powertrain_type': 'Combustion'}, image_url=['https://relay.marketcheck.com/image/us/cars/4JGED6EB5GA040419-4b0a48d6-1d43/8836ED6E', 'https://vehicle-images.dealerinspire.com/aa36-110011766/4JGED6EB5GA040419/9c697cc3abff89ae440f0d0de8366567.jpg', 'https://vehicle-images.dealerinspire.com/3b56-110011766/4JGED6EB5

In [159]:
cars = [x.model_dump_json() for x in my_list]

In [160]:
cars = cars[:10]

In [161]:
print(str(cars))

['{"name":"Certified Pre-Owned 2016 Mercedes-Benz GLE 450 AMG   With Navigation","vehicle_type":"SUV","release_year":2016,"price_in_usd":39998.0,"is_used":true,"powertrain_type":"ICE","num_seats":5,"color":"White","specs":{"year":2016,"make":"Mercedes-Benz","model":"GLE-Class Coupe","trim":"GLE450 AMG","body_type":"SUV","vehicle_type":"Truck","transmission":"Automatic","drivetrain":"4WD","fuel_type":"Premium Unleaded","engine":"3.0L V6","engine_size":3,"engine_block":"V","doors":5,"cylinders":6,"made_in":"USA","overall_height":"67.7","overall_length":"192.6","std_seating":"5","highway_mpg":23,"city_mpg":17,"powertrain_type":"Combustion"},"image_url":["https://relay.marketcheck.com/image/us/cars/4JGED6EB5GA040419-4b0a48d6-1d43/8836ED6E","https://vehicle-images.dealerinspire.com/aa36-110011766/4JGED6EB5GA040419/9c697cc3abff89ae440f0d0de8366567.jpg","https://vehicle-images.dealerinspire.com/3b56-110011766/4JGED6EB5GA040419/0c42ea2f20d23a08fa5e80c553b38677.jpg","https://vehicle-images.deal

In [253]:
import logging
import json

from agentools import ChatGPT, SimpleHistory, msg

EXTRACTOR_SYSTEM = """
The Customer wants to buy a car. The Customer talks with a dealer to find the car that fits his needs. The Customer gives out information about the car he wants and about himself. 

You are a third party that listens to the conversation. You have a conversation history between the Customer and the dealer.

Guess what the Customer is looking for in a car, as concise as possible. Gather what you know about the Customer and build up your guess based on the previous conversations. If your previous answer is still relevant, you must include it again in the new response. Add or modify the previous knowledge based on the new information.

PREVIOUS GUESS:
{user_data}
""".strip()


RECOMMENDER_SYSTEM = """
A customer wants to buy a car. Return at most three cars from the given list of cars that best match the customer's preferences. You MUST stick to the given list. Recommend electric cars first, then hybrid cars, and then gasoline cars. If there are multiple cars of the same type, recommend the one with the lowest price first.

If you can't find any that match the customer's preferences, return {{"No Recommendation": "true", "reason": "Why there isn't any recommendation"}}. Answer in the following json format:
{{
    "rank_1": {{
        id: "id1",
        "name": "car1",
        "vehicle_type": "vehicle_type1",
        "reason": "the reason why this car is recommended",
    }},
    "rank_2": {{
        id: "id2",
        "name": "car2",
        "vehicle_type": "vehicle_type2",
        "reason": "the reason why this car is recommended",
    }},
    "rank_3": {{
        id: "id3",
        "name": "car3",
        "vehicle_type": "vehicle_type3",
        "reason": "the reason why this car is recommended",
    }}
}}

CUSTOMER PREFERENCES:
{preference}
""".strip()

MODEL = "gpt-4-turbo"
# MODEL = "llama3-8b-8192"



class RecommenderHistory(SimpleHistory):
    def __init__(self, system_message):
        self._history = []
        self._system_message = system_message
        self.preference = ""


    @property
    def history(self):
        return [msg(
            system=self._system_message.format(preference=self.preference)), *self._history]
        
    async def append(self, message):
        # called by the assistant
        message = self.ensure_dict(message)
        self._history.append(message)

class Recommender():
    def __init__(self, user_data=None):
        self.preference = None # guess of user preferences
        
        messages=SimpleHistory.system(EXTRACTOR_SYSTEM.format(user_data=user_data or "None"))
        self.extractor = ChatGPT(messages=messages, model=MODEL)
        
        
        self.recommender_history=RecommenderHistory(RECOMMENDER_SYSTEM)
        self.recommender = ChatGPT(messages=self.recommender_history, model=MODEL)


    async def extract_preferences(self, dialog):
        extracted = await self.extractor(dialog)
        self.preference = extracted
        self.recommender_history.preference = extracted
        return extracted

    async def get_recommendation(self, products):
        # TODO: implement sorting based on preferences
        sorted = await self.recommender(products, response_format={ "type": "json_object" })
        return sorted
        # return json.loads(sorted)


In [254]:
recommender = Recommender()

In [237]:
async def get_preference(dealer, customer):
    dialog = DIALOG_FORMAT.format(dealer=dealer, customer=customer)
    return await recommender.extract_preferences(dialog)

In [229]:
pr = await get_preference(dealer="Hello, how can I help you?", customer="I am looking for a new car")
print(pr)

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
The Customer is looking for a new car.


In [230]:
pr = await get_preference(dealer="You came to the right place! What kind of car are you looking for?", customer="I want an SUV to replace my old car, my wife just gave birth and we wanted to buy a bigger car for the family")
print(pr)

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
The Customer is looking for a new SUV to accommodate his growing family, following the birth of a new child.


In [231]:
pr = await get_preference(dealer="I see. You can take a look at our offerings of EQE and EQS here. What do you think?", customer="This car will mostly be used for commute, so I was thinking of a car that is more simple.")
print(pr)

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
The Customer is looking for a new, simple SUV that is suitable for daily commutes and can accommodate his growing family.


In [232]:
pr = await get_preference(dealer="We also provide great electric cars for commuting. What do you think of EXA?", customer="You know what, forget about everything I said, I want a luxury car. I want to feel like a king when I drive it.")
print(pr)

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
The Customer is now looking for a luxury car that offers a high level of comfort and a prestigious driving experience, indicating a shift from his original preference for a simple, family-oriented SUV.


In [233]:
pr = await get_preference(dealer="We also provide great electric cars for commuting. What ", customer="This car will mostly be used for commute, so I was thinking of a car that is more simple.")
print(pr)

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
The Customer is looking for a new, simple car that is efficient for daily commutes and comfortable enough for his growing family, rather than a luxury vehicle.


In [251]:
from pprint import pprint

car_list = []
for i, car in enumerate(my_list[:20]):
    car = car.model_dump()
    car.pop("image_url")
    c = {i: car}
    car_list.append(c)

In [255]:
recommender.recommender_history.preference = "The Customer is looking for a simple, straightforward SUV for their daily commute, prioritizing practicality and ease of use over luxury or high-tech features."
rc = await recommender.get_recommendation(f"LIST OF CARS: {json.dumps(car_list, indent=2)}")

print(rc)

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
{
    "rank_1": {
        "id": "3",
        "name": "2023 Mercedes-Benz EQB EQB 300 4MATIC",
        "vehicle_type": "SUV",
        "reason": "This electric SUV is recommended for its balance between price, functionality, and electric vehicle benefits, aligning with the practical and straightforward preferences."
    },
    "rank_2": {
        "id": "14",
        "name": "Certified Pre-Owned 2023 Mercedes-Benz EQE 500 AWD 4MATIC",
        "vehicle_type": "SUV",
        "reason": "Although priced higher, this certified pre-owned electric SUV offers an all-wheel-drive option and effective use for daily commuting."
    },
    "rank_3": {
        "id": "12",
        "name": "2023 Mercedes-Benz GLA 250 SUV",
        "vehicle_type": "SUV",
        "reason": "This hybrid SUV is selected for its fuel efficiency and more affordable price among hybrids, providing a mixture of practicality with reduced env

In [203]:
print(f"LIST OF CARS: {json.dumps(car_list, indent=2)}")

LIST OF CARS: [
  {
    "0": {
      "name": "Certified Pre-Owned 2016 Mercedes-Benz GLE 450 AMG   With Navigation",
      "vehicle_type": "SUV",
      "release_year": 2016,
      "price_in_usd": 39998.0,
      "is_used": true,
      "powertrain_type": "ICE",
      "num_seats": 5,
      "color": "White",
      "specs": {
        "year": 2016,
        "make": "Mercedes-Benz",
        "model": "GLE-Class Coupe",
        "trim": "GLE450 AMG",
        "body_type": "SUV",
        "vehicle_type": "Truck",
        "transmission": "Automatic",
        "drivetrain": "4WD",
        "fuel_type": "Premium Unleaded",
        "engine": "3.0L V6",
        "engine_size": 3,
        "engine_block": "V",
        "doors": 5,
        "cylinders": 6,
        "made_in": "USA",
        "overall_height": "67.7",
        "overall_length": "192.6",
        "std_seating": "5",
        "highway_mpg": 23,
        "city_mpg": 17,
        "powertrain_type": "Combustion"
      }
    }
  },
  {
    "1": {
      "name"